In [56]:
import pandas as pd


ImportTable1 = "../Outputs/RawWine.csv"
wine = pd.read_csv(ImportTable1, encoding="UTF-8")

ImportTable = "../Outputs/state.csv"
state = pd.read_csv(ImportTable, encoding="UTF-8")

ImportTable = "../Outputs/Taster.csv"
taster = pd.read_csv(ImportTable, encoding="UTF-8")


In [54]:
taster.head()

,Unnamed: 0,tasterID,taster_name,taster_twitter_handle
0,0,1,Anne Krebiehl MW,@AnneInVino
1,1,2,Christina Pickard,@winewchristina
2,2,3,Fiona Adams,@bkfiona
3,3,4,Jeff Jenssen,@worldwineguys
4,4,5,Jim Gordon,@gordone_cellars


In [58]:
wine.head()

,Unnamed: 0,country,points,price,province,title,variety,taster_name
0,0,Italy,87,NaN,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Kerin O’Keefe
1,1,Portugal,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Roger Voss
2,2,US,87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Paul Gregutt
3,3,US,87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,Alexander Peartree
4,4,US,87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Paul Gregutt


In [59]:
#wine_join_state.head()
wine = wine.rename( columns={"province": "state"})
#state = state.set_index(inplace=True)
state.drop_duplicates("state", inplace=True)
state.head()

,Unnamed: 0,stateId,state,GDP,countryId
0,0,1,Douro,0,31
1,1,2,Oregon,"238,687",39
2,2,3,Northern Spain,0,36
3,3,4,Sicily & Sardinia,0,22
4,4,5,Alsace,0,15


In [63]:
RawWineTable = pd.merge(wine_join_state, state, on="state", how ="left")
WineTable = pd.merge(RawWineTable, taster, on="taster_name")
WineTable.head()

,Unnamed: 0_x,country,points,price,state,title,variety,taster_name,Unnamed: 0_y,stateId,GDP,countryId,Unnamed: 0,tasterID,taster_twitter_handle
0,0,Italy,87,NaN,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Kerin O’Keefe,3.0,4.0,0,22.0,6,7,@kerinokeefe
1,6,Italy,87,16.0,Sicily & Sardinia,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Kerin O’Keefe,3.0,4.0,0,22.0,6,7,@kerinokeefe
2,13,Italy,87,NaN,Sicily & Sardinia,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Kerin O’Keefe,3.0,4.0,0,22.0,6,7,@kerinokeefe
3,22,Italy,87,19.0,Sicily & Sardinia,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Kerin O’Keefe,3.0,4.0,0,22.0,6,7,@kerinokeefe
4,24,Italy,87,35.0,Sicily & Sardinia,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Kerin O’Keefe,3.0,4.0,0,22.0,6,7,@kerinokeefe


In [64]:
WineTable.reset_index(inplace=True)
WineTable["wineId"] = WineTable["index"]+1

In [65]:
WineTabletoSQL=WineTable[["wineId","title","variety","points","price","stateId","tasterID" ]]
WineTabletoSQL = WineTabletoSQL.rename( columns={"tasterID": "testerId"})
WineTabletoSQL=WineTabletoSQL
WineTabletoSQL.head()

,wineId,title,variety,points,price,stateId,testerId
0,1,Nicosia 2013 Vulkà Bianco (Etna),White Blend,87,NaN,4.0,7
1,2,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,87,16.0,4.0,7
2,3,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,87,NaN,4.0,7
3,4,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,87,19.0,4.0,7
4,5,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,87,35.0,4.0,7


In [7]:
from sqlalchemy import create_engine

In [66]:

rds_connection_string = "root:Sanangel1@127.0.0.1/Wine_Ratings"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [25]:
engine.table_names()

['country', 'state', 'taster', 'wine']

In [67]:
WineTabletoSQL.to_sql(name='wine', con=engine, if_exists='append', index=False)

In [68]:
pd.read_sql_query('select * from wine', con=engine).head()

,wineId,title,variety,points,price,stateId,testerId
0,1,Nicosia 2013 Vulkà Bianco (Etna),White Blend,87,NaN,4.0,7
1,2,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,87,16.0,4.0,7
2,3,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,87,NaN,4.0,7
3,4,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,87,19.0,4.0,7
4,5,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,87,35.0,4.0,7
